# Instalação
#INSTALAÇÃO DAS LIBS
- !pip install yfinance==0.2.41
- !pip install crewai==0.28.8
- !pip install 'crewai[tools]'
- !pip install langchain==0.1.20
- !pip install langchain-openai==0.1.7
- !pip install langchain-community==0.0.38
- !pip install duckduckgo-search==5.3.0

In [17]:
import json
import os

from datetime import datetime

import yfinance as yf

from crewai import Agent, Task

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

In [11]:
# Criando yahoo finance tool
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock


yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
    func=lambda ticket: fetch_stock_price(ticket)
)

In [12]:
print(fetch_stock_price('AAPL'))

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643402   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043762   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825180   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546387   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date         

In [22]:
import os

# Importando OpenAI LLM - GPT
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o")

In [14]:
stockPriceAnalyst = Agent(
    role="Senior stock price Analyst",
    goal="Find the {ticket} stock and analyses treds",
    backstory="""You are a highly experienced in analyzing the price of an specif stock
    and make predictions about its future price.""",
    verbose=True,
    llm=llm,
    max_iter=5,
    tools=[yahoo_finance_tool]
    )

In [15]:
getStockPrice = Task(
    description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output=""" Specify the current trend stock price -up, down or sideways.
    eg. stock 'APPL, price UP'
    """,
    agent=stockPriceAnalyst
)

In [18]:
# Important a tool de search

search_tool = DuckDuckGoSearchResults(
    backend='news',
    num_results=10
)

In [19]:
newAnalyst = Agent(
    role="Stock News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways
     with the news context. For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
    backstory=""" You are a highly experienced in analyzing the market trends and news and have tracked assets for more then 10 years.
    
    You are also master level alyts in the tradicional markets and have deep understanding of human psychology.
    
    YOu understand news, theirs tittles and information, but you look at those with a health dose of skepticise.
    You consider also the source of the news articles.
    """,
    verbose=True,
    llm=llm,
    max_iter=10,
    tools=[search_tool]
)

In [20]:
get_news = Task(
    description="""Take the stock and always include BTC to it (if not request).env
    Use the search tool to search each one individually.
    
    the current date is {datetime.now()}.
    
    Compose the results into a helpfull report.""",
    expected_output="""A summary of the overall market and one sentence summary for each request asset.
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
    agent=newAnalyst
)

In [21]:
stockAnalystWrite = Agent(
    role="Senior Stock Analyts Writer",
    goal="""Analyze the trends price, news and write an insighfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend.""",
    backstory="""You are widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories
    and narratives that resonate with wider audiences.
    
    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses. you are able to hold multpli opinions when analyzing anything.
    """
)